# Actividad 5:
# Aplicación de Regularización en Modelo de Regresión

## Objetivo
Aplicar técnicas de regularización (Lasso, Ridge, Elastic Net) en un modelo re regresión y comparar su rendimiento en términos generalización y selección de características.

**Datasets utilizados:**  
`Adult Income`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y preprocesamiento de datos:**
  - Se carga el dataset **Adult Income** y se hace una limpieza de datos, utilizando un 5% del total de datos como umbral para borrar o imputar los datos faltantes o que presenten problemas.

2. **Evaluación y análisis:**
  - Técnicas utilizadss:
    - Lasso.
    - Ridge.
    - Elastic Net.
  - Se uso además optimización con optuna para encontrar los mejores hiperparametros de cada técnica.

  - Métricas empleadas:
    - MSE
    - RMSE

3. **Visualización de resultados:**
  - Tabla resumen de las mejores variables de cada modelo.
  - Tabla comparativa de las tres técnicas con MSE y RMSE.
  - Gráficos de top 10 variables por modelo con sus coeficientes.

---

# 2. Configuración del entorno

--- 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import optuna
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


# Silenciar logs de Optuna y warnings generales
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")

# Estilo gráfico y opciones de pandas
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos y entrenamiento de modelos.

- **`carga_y_preprocesamiento()`** 
Carga y prepara el dataset Adult Income, manejando valores faltantes, codificando y escalando variables, y dividiendo en train/test.

In [ ]:
def carga_y_preprocesamiento():
    """
    Carga y preprocesa el dataset 'Adult Income' de OpenML para clasificación binaria.

    Pasos realizados:
    - Carga el dataset 'adult' versión 2.
    - Reemplaza valores '?' por NaN para el manejo de datos faltantes.
    - Analiza y reporta porcentaje de valores faltantes por columna.
    - Elimina columnas con más del 5% de datos faltantes.
    - Imputa valores faltantes en columnas numéricas con la mediana.
    - Imputa valores faltantes en columnas categóricas con la moda.
    - Codifica la variable objetivo 'class' a binaria: 1 si ingreso >50K, 0 en caso contrario.
    - Divide el dataset en conjuntos de entrenamiento (70%) y prueba (30%), manteniendo la proporción de clases (stratify).
    - Aplica preprocesamiento con pipelines:
        * Imputación y escalado (StandardScaler) para variables numéricas.
        * Codificación One-Hot para variables categóricas.
    - Retorna los datos preprocesados para entrenamiento y prueba.

    Returns:
        X_train_prep (np.ndarray): Matriz preprocesada de características para entrenamiento.
        X_test_prep (np.ndarray): Matriz preprocesada de características para prueba.
        y_train (pd.Series): Vector objetivo binario para entrenamiento.
        y_test (pd.Series): Vector objetivo binario para prueba.

    """
    # Cargar dataset
    adult = fetch_openml("adult", version=2, as_frame=True)
    df = adult.frame
    
    print("Distribución de clases original:")
    print(df['class'].value_counts(normalize=True))
    
    # Manejar valores faltantes
    df.replace('?', np.nan, inplace=True)
    
    # Analizar valores faltantes
    missing_percent = df.isna().mean().sort_values(ascending=False)
    print("\nPorcentaje de valores faltantes por columna:")
    print(missing_percent[missing_percent > 0])
    
    # Eliminar columnas con >10% de valores faltantes
    threshold = 0.1
    cols_to_drop = missing_percent[missing_percent > threshold].index
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    # Variable objetivo binaria
    y = (df['class'] == '>50K').astype(int)
    X = df.drop('class', axis=1)
    
    # Dividir en train/test (70/30)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Preprocesamiento
    num_cols = X_train.select_dtypes(include=['int', 'float']).columns
    cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
    
    # Crear pipeline para variables numéricas
    num_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # Codificación para variables categóricas
    cat_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # ColumnTransformer combinando ambos
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ])

    X_train_prep = preprocessor.fit_transform(X_train)
    X_test_prep = preprocessor.transform(X_test)

    # Obtener nombres de las features tras preprocesamiento
    # Nombres numéricos (sin cambio)
    feature_names_num = list(num_cols)
    # Nombres categóricos (OneHotEncoder)
    cat_encoder = preprocessor.named_transformers_['cat']['encoder']
    feature_names_cat = list(cat_encoder.get_feature_names_out(cat_cols))
    # Concatenar nombres
    feature_names = feature_names_num + feature_names_cat
    
    print(f"\nDimensiones después de preprocesamiento:")
    print(f"Train: {X_train_prep.shape}, Test: {X_test_prep.shape}")
    
    return X_train_prep, X_test_prep, y_train, y_test, feature_names, preprocessor # se guarda preprocessor por si se usara a futuro en un entorno de producción

**Bloque 2:** Optimización y evaluación de las tres técnicas de regularización.

- **`optimizar_y_evaluar()`** 
Optimiza con optuna cada técnica (Lasso, Ridge y ElasticNet) y entrena cada modelos con los mejores hiperparametros obtenidos de optuna.

In [ ]:
def optimizar_y_evaluar(X_train, y_train, X_test, y_test, feature_names, n_trials=10):

    def objective(trial, model_name):
        if model_name == 'Lasso':
            alpha = trial.suggest_float('alpha', 1e-3, 1)
            model = Lasso(alpha=alpha, max_iter=10000)
        elif model_name == 'Ridge':
            alpha = trial.suggest_float('alpha', 1e-3, 1)
            model = Ridge(alpha=alpha, max_iter=10000)
        else:  # ElasticNet
            alpha = trial.suggest_float('alpha', 1e-3, 1)
            l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
            model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=10000)

        score = -cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
        return score

    resultados = {}

    for modelo_nombre in ['Lasso', 'Ridge', 'ElasticNet']:
        print(f"\nOptimizando {modelo_nombre}...")
        study = optuna.create_study(direction='minimize')
        func_obj = lambda trial: objective(trial, modelo_nombre)
        study.optimize(func_obj, n_trials=n_trials)
        
        print(f"Mejores parámetros para {modelo_nombre}: {study.best_params}")
        
        # Entrenar con mejores hiperparámetros
        if modelo_nombre == 'Lasso':
            model = Lasso(alpha=study.best_params['alpha'], max_iter=10000)
        elif modelo_nombre == 'Ridge':
            model = Ridge(alpha=study.best_params['alpha'], max_iter=10000)
        else:
            model = ElasticNet(alpha=study.best_params['alpha'],
                               l1_ratio=study.best_params['l1_ratio'], max_iter=10000)
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        coefs = pd.Series(model.coef_, index=feature_names)
        coefs_importantes = coefs.abs().sort_values(ascending=False).head(10)
        
        print(f"MSE test {modelo_nombre}: {mse:.4f}")
        print("Top 10 variables importantes:")
        print(coefs_importantes)
        
        resultados[modelo_nombre] = {
            'mse': mse,
            'coeficientes': coefs,
            'best_params': study.best_params
        }
    
    return resultados

**Bloque 3:** Visualización de resultados.

- **`mostrar_tabla_mse()`** 
Muestra una tabla comparativa del MSE y adicional el RMSE de cada técnica.

- **`mostrar_tabla_hiperparametros()`** 
Muestra una tabla de los mejores hiperparámetros para cada modelo (alpha para Lasso y Ridge, alpha y l1_ratio para ElasticNet).

- **`plot_top_10_variables()`** 
Grafica las 10 variables mas importantes por cada modelo.

In [ ]:
def mostrar_tabla_mse(resultados):
    """
    Genera y muestra una tabla comparativa de MSE y RMSE para los modelos evaluados.
    
    Parámetros:
        resultados (dict): Diccionario retornado por `optimizar_y_evaluar`
    """
    tabla = []

    for modelo, valores in resultados.items():
        mse = valores['mse']
        rmse = np.sqrt(mse)
        tabla.append({'Modelo': modelo, 'MSE': mse, 'RMSE': rmse})

    df_resultados = pd.DataFrame(tabla).set_index("Modelo")
    print("\nResumen de errores (MSE y RMSE):")
    display(df_resultados.round(4))
    
    return df_resultados

def mostrar_tabla_hiperparametros(resultados):
    tabla = []
    for modelo, valores in resultados.items():
        params = valores['best_params']
        fila = {'Modelo': modelo}
        fila.update(params)  # Añade cada hiperparámetro como columna
        tabla.append(fila)
    
    df_hiper = pd.DataFrame(tabla).set_index("Modelo")
    print("\nResumen de mejores hiperparámetros:")
    display(df_hiper.round(6))
    
    return df_hiper


def plot_top_10_variables(resultados):
    """
    Genera una figura con 3 subgráficos, cada uno mostrando las 10 variables
    más importantes (por valor absoluto del coeficiente) para Lasso, Ridge y ElasticNet.
    
    Parámetros:
        resultados (dict): Diccionario retornado por `optimizar_y_evaluar`
    """
    modelos = ['Lasso', 'Ridge', 'ElasticNet']
    fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

    for i, modelo in enumerate(modelos):
        coefs = resultados[modelo]['coeficientes']
        top10 = coefs.abs().sort_values(ascending=False).head(10)
        top10.plot(kind='barh', ax=axes[i], color='purple', edgecolor='black')
        axes[i].set_title(f'{modelo}: Top 10 variables')
        axes[i].invert_yaxis()
        axes[i].set_xlabel('Importancia (|coef|)')

    plt.tight_layout()
    plt.show()

**Bloque 4:** Función de ejecución del código.

- **`main()`** 
Ejecuta todo el flujo: carga datos, optimiza, entrena y evalúa los tres modelos de técnicas de regularización, y muestra los resultados.

In [ ]:
def main():
    X_train_prep, X_test_prep, y_train, y_test, feature_names, preprocessor = carga_y_preprocesamiento()

    resultados = optimizar_y_evaluar(X_train_prep, y_train, X_test_prep, y_test, feature_names, n_trials=20)

    # Variables eliminadas por Lasso
    coefs_lasso = resultados['Lasso']['coeficientes']
    eliminadas_lasso = coefs_lasso[coefs_lasso == 0].index.tolist()
    print(f"\nNúmero de variables eliminadas por Lasso: {len(eliminadas_lasso)}")
    print(pd.DataFrame({'Variable eliminada por Lasso': eliminadas_lasso}))

    # Variables eliminadas por ElasticNet
    coefs_enet = resultados['ElasticNet']['coeficientes']
    eliminadas_enet = coefs_enet[coefs_enet == 0].index.tolist()
    print(f"\nNúmero de variables eliminadas por ElasticNet: {len(eliminadas_enet)}")
    print(pd.DataFrame({'Variable eliminada por ElasticNet': eliminadas_enet}))

    tabla_hp = mostrar_tabla_hiperparametros(resultados)
    tabla_mse = mostrar_tabla_mse(resultados)

    plot_top_10_variables(resultados)


# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# Análisis de Técnicas de Regularización

> **Nota:** Los resultados expuestos se basan en una ejecución específica. Al re-ejecutar el código, los valores podrían variar ligeramente, ya que la optimización se realiza mediante búsquedas aleatorias (Optuna). Las diferencias pueden afectar cuál modelo obtiene el menor error o cuántas variables son eliminadas, especialmente entre Lasso y ElasticNet.

## ¿Cuál de las técnicas de regularización (Lasso, Ridge o ElasticNet) fue más efectiva para este conjunto de datos?

En términos de rendimiento en el conjunto de test, el modelo **Ridge** obtuvo el menor error cuadrático medio (MSE = **0.1158**), seguido por **ElasticNet** (MSE = **0.1172**) y finalmente **Lasso** (MSE = **0.1211**). Esto sugiere que:

- **Ridge** fue el más efectivo, logrando el mejor desempeño general.
- **ElasticNet** tuvo un rendimiento competitivo, combinando las fortalezas de Lasso y Ridge.
- **Lasso**, aunque con mayor MSE, simplificó fuertemente el modelo eliminando muchas variables.

| Modelo      | MSE     | RMSE   |
|-------------|---------|--------|
| Lasso       | 0.1211  | 0.3479 |
| Ridge       | 0.1158  | 0.3403 |
| ElasticNet  | 0.1172  | 0.3423 |

---

## ¿Qué variables fueron eliminadas por Lasso y ElasticNet, y por qué?

### Lasso eliminó **98** de las 105 variables (~93.3%)
Esto incluye principalmente:
- Variables de `native-country` (ej.: Yugoslavia, Vietnam, Thailand).
- Variables de `workclass` (ej.: Never-worked, Private, Local-gov).
- Otras variables categóricas poco frecuentes o con baja correlación con el objetivo.

### ElasticNet eliminó **75** variables (~71.4%)
- Mantuvo más variables que Lasso, pero también eliminó muchas relacionadas con `native-country` y `workclass`.

Esto se debe a que:
- **Lasso** aplica una penalización L1, que fuerza a cero los coeficientes de variables menos relevantes.
- **ElasticNet** también puede forzar coeficientes a cero, dependiendo del valor de `l1_ratio`.
- Ambas técnicas ayudan a seleccionar un subconjunto más informativo de variables, especialmente útil en datasets con alta dimensionalidad tras one-hot encoding.

> **Conclusión parcial:** Lasso y ElasticNet simplifican el modelo al eliminar variables irrelevantes, mejorando la interpretabilidad sin sacrificar demasiado rendimiento.

---

## ¿Cómo impactó la regularización en la complejidad del modelo y su capacidad para generalizar?

### Reducción de complejidad:
- Lasso redujo drásticamente el número de variables activas, facilitando la interpretación.
- ElasticNet también redujo la complejidad, aunque en menor grado.
- Ridge conservó todas las variables, pero limitó sus coeficientes para evitar sobreajuste.

### Generalización:
- La regularización ayudó a evitar el overfitting.
- Ridge y ElasticNet lograron mejor MSE en test que Lasso, lo que indica mayor capacidad de generalización.

### Interpretabilidad vs rendimiento:
- Lasso ofrece el mejor compromiso en cuanto a interpretabilidad.
- Ridge prioriza el rendimiento, ideal cuando la precisión es lo más importante.
- ElasticNet se ubica en un punto intermedio, manteniendo buen rendimiento y cierta capacidad de simplificación.

---

## Conclusión

La regularización fue clave para manejar la alta dimensionalidad del dataset tras aplicar one-hot encoding.

- **Ridge** fue el modelo con mejor rendimiento general.
- **Lasso** destacó por su capacidad de reducir el número de variables a solo las más relevantes.
- **ElasticNet** ofreció un equilibrio entre rendimiento y simplificación.

Estas técnicas permitieron construir modelos más robustos y con mejor capacidad de generalización, siendo fundamentales en problemas con muchas variables y correlaciones entre ellas.